# Imports

In [1297]:
import pandas as pd
import numpy as np
import itertools

# import matplotlib.pyplot as plt
# import seaborn as sns
# import warnings
# import joblib

# import psycopg2 as ps

# from sklearn.base import BaseEstimator, TransformerMixin
# from sklearn.preprocessing import MinMaxScaler, OneHotEncoder, OrdinalEncoder
# from sklearn.pipeline import Pipeline

# from sklearn.model_selection import train_test_split
# from sklearn.metrics import accuracy_score, recall_score, precision_score, confusion_matrix, ConfusionMatrixDisplay

import os

In [1298]:
PATH_IGNORE = 'ignore'
PATH_DATA = 'data'

FILE_DATA = os.path.join(PATH_DATA, 'BASE DE DADOS PEDE 2024 - DATATHON.xlsx')
FILE_OLD_DATA = os.path.join(PATH_DATA, 'PEDE_PASSOS_DATASET_FIAP_ANTIGO.xlsx')
FILE_JOINED_DATA = os.path.join(PATH_DATA, 'data_joined.csv')
FILE_JOINED_DATA2022 = os.path.join(PATH_DATA, 'data_joined_2022.csv')
FILE_JOINED_DATA2023 = os.path.join(PATH_DATA, 'data_joined_2023.csv')
FILE_JOINED_DATA2024 = os.path.join(PATH_DATA, 'data_joined_2024.csv')

FILE_POSTGRES_PW = os.path.join(PATH_IGNORE, 'postgres-pw.txt')

In [1299]:
raw_data = {}

raw_data[2022] = pd.read_excel(FILE_DATA, sheet_name=0)
raw_data[2023] = pd.read_excel(FILE_DATA, sheet_name=1)
raw_data[2024] = pd.read_excel(FILE_DATA, sheet_name=2)

# Dicionário

**INDE** - Índice de desenvolvimento educacional
- Ponderação entre os indicadores abaixo

**IAN** - Indicador de adequação de nível
- Registros administrativos
- Registra a defasagem da aprendizagem do aluno por meio da comparação da Fase atual do aluno da Associação, com a equivalência das Fases de ensido na Associação e a divisão dos anos escolares do ensino formal (por idade)
- Utiliza a fase atual, e a fase ideal conforme a idade.
- 10 (em fase), 5 (moderada) ou 2,5 (severa).

**IDA** - Indicador de desempenho acadêmico
- Notas Provas PM e Média geral Universitária
- Registra a proficiência do aluno nos exames padronizados de avaliação interna das disciplinas oferecidas pela Associação (os alunos das Fases 0 a 7). No caso dos alunos da Fase 8, esse indicador registra a nota média obtida pelos alunos em todas as disciplinas curriculares cursadas, nas respectivas instituições de ensino superior conveniadas ao programa de bolsas de estudo da Associação.
- Utiliza as notas internas da associação. Matemática + Português + Inglês / 3
- 0 a 10

**IEG** - Indicador de engajamento
- Registros de entrega de lição de casa e de voluntariado
- Mede o engajamento do aluno nas tarefas curriculares requeridas em cada uma das disciplinas oferecidas na Associação, e/ou seu engajamento em ações de voluntariado desenvolvidas pela Associação.
- Soma das pontuações das tarefas realizadas e registradas / Número de tarefas

**IAA** - Indicador de Autoavaliação
- Questionário de Autoavaliação individual
- Registra por meio de um questionário padronizado e adaptado às distintas faixas etárias dos alunoa a Associação, uma autoavaliação do aluno sobre como se sente consigo mesmo, sobre os estudos, sobre sua família, amigos e comunidade, e e sobre como se sente a respeito da Associação Passos Mágicos.
- Soma das pontuações das respostas do estudante / Número total de perguntas

**IPS** - Indicador Psicossocial
- Questionário individual de avaliação das psicólogas
- Avaliação da equipe de psicólogas para caracterizar o desenvolvimento do aluno nas suas interações familiares, no seu desenvolvimento emocional, comportamental e da sua socialização na vida comunitária. Esse indicador também caracteriza o tipo de atendimento psicológico oferecido pela Associação ao aluno.
- Avaliações feitas por **psicólogos**.
- Soma das pontuações / Número de avaliadores

**IPP** - Indicador Psicopedagógico
- Questionário individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos para caracterizar o desenvolvimento cognitivo, emocional, comportamental e de socialização do aluno no seu processo de aprendizado dentro do Programa de Aceleração do Conhecimento, dos Programas Educacionais e das Atividades Culturais promovidas pela Associação.
- Soma das avaliações sobre aspectos **pedagógicos** / Número de avaliações

**IPV** - Indicador de Ponto de Virada
- Questionário Individual de avaliação dos pedagogos e professores
- Avaliação da equipe de educadores e psicopedagogos a respeito do desenvolvimento do aluno das aptidões necessárias para iniciar a transformação da sua vida por meio da Educação, avaliando a integração do aluno à Associação, o seu desenvolvimento emocional, e o seu potencial acadêmico.
- Análises longitudinais de progresso acadêmico, engajamento e desenvolvimento emocional

**Pedras** - Baseadas no INDE
- Quartzo: 3.0 < INDE < 6.1
- Ágata: 6.1 < INDE < 7.2
- Ametista: 7.2 < INDE < 8.2
- Topázio: 8.2 < INDE < 9.4

# Tratando dados
## Padronizando colunas
As colunas são inconsistente entre os datasets. Vamos extrair os dados que queremos um por um, ano por ano.

In [1300]:
dfs_yearly = {}

dfs_yearly[2022] = pd.DataFrame()
dfs_yearly[2023] = pd.DataFrame()
dfs_yearly[2024] = pd.DataFrame()

In [1301]:
def display_column_from_all(
    col : str,
    head : int = 3
):
    for df in dfs_yearly.values():
        display(df[col].head(head))

In [1302]:
def display_value_counts_from_all(
	col : str
):
	for df in dfs_yearly.values():
		print(df[col].value_counts())

In [1303]:
def replace_value_with(
	df,
	col,
	value,
	replace_with = np.nan
):
    to_na = df[col] == value # mask
    df.loc[to_na, col] = replace_with    

In [1304]:
def extract_data(
    col_name : str,
    col_2022 : str,
    col_2023 : str,
    col_2024 : str
):
	# 2022
	year = 2022
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2022]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

	# 2023
	year = 2023
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2023]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

	# 2024
	year = 2024
	try:
		dfs_yearly[year][col_name] = raw_data[year][col_2024]
	except Exception as e:
		print(f'Failed to find column on {year} | {e}')
		dfs_yearly[year][col_name] = np.nan

In [1305]:
def process_text(
	df,
	col
):
	try:
		df[col] = df[col].str.capitalize()
	except Exception as e:
		print(f'Failed to process text on df[{col}] | {e}')

In [1306]:
def process_text_on_all_years(
        col
):
    for year in dfs_yearly:
        df = dfs_yearly[year]
        process_text(df, col)

### Alunos

In [1307]:
col = 'aluno'

extract_data(
	col,
	'Nome',
	'Nome Anonimizado',
	'Nome Anonimizado'
)

In [1308]:
display_column_from_all(col)

0    Aluno-1
1    Aluno-2
2    Aluno-3
Name: aluno, dtype: object

0    Aluno-861
1    Aluno-862
2    Aluno-863
Name: aluno, dtype: object

0    Aluno-1275
1    Aluno-1276
2    Aluno-1277
Name: aluno, dtype: object

### Idade

In [1309]:
col = 'idade'

extract_data(
	col,
	'Idade 22',
	'Idade',
	'Idade'
)

In [1310]:
display_column_from_all(col)
display_value_counts_from_all(col)

0    19
1    17
2    17
Name: idade, dtype: int64

0    8
1    9
2    7
Name: idade, dtype: object

0    8
1    8
2    8
Name: idade, dtype: int64

idade
10    116
12    107
11    102
13    100
9      96
14     85
15     63
8      57
16     52
17     48
7      14
18     12
19      5
20      2
21      1
Name: count, dtype: int64
idade
11                     77
10                     77
13                     65
9                      65
12                     62
1900-01-11 00:00:00    56
8                      55
14                     54
1900-01-12 00:00:00    47
1900-01-09 00:00:00    43
15                     43
1900-01-14 00:00:00    42
1900-01-10 00:00:00    36
16                     36
1900-01-08 00:00:00    34
1900-01-13 00:00:00    30
17                     25
1900-01-15 00:00:00    22
1900-01-17 00:00:00    21
1900-01-16 00:00:00    19
18                     18
1900-01-07 00:00:00    14
7                      13
1900-01-18 00:00:00    11
1900-01-19 00:00:00    10
1900-01-20 00:00:00    10
19                      8
21                      6
20                      5
22                      4
1900-01-21 00:00:00     3
1900-0

In [1311]:
is_int = pd.Series()


count = 0

for row in dfs_yearly[2023][col]:
    is_int[count] = str(type(row)) == "<class 'int'>"
    count += 1

is_int.index = dfs_yearly[2023].index

In [1312]:
dfs_yearly[2023].loc[is_int == False, 'idade'] = pd.to_datetime(dfs_yearly[2023].loc[is_int == False, 'idade']).dt.day
dfs_yearly[2023].loc[is_int == False, 'idade']

3        8
5        7
7        7
12      11
15       8
        ..
1000    17
1005    18
1006    21
1011    18
1013    18
Name: idade, Length: 399, dtype: object

In [1313]:
dfs_yearly[2023]['idade'] = dfs_yearly[2023]['idade'].astype(int)

In [1314]:
display_column_from_all(col)
display_value_counts_from_all(col)

0    19
1    17
2    17
Name: idade, dtype: int64

0    8
1    9
2    7
Name: idade, dtype: int64

0    8
1    8
2    8
Name: idade, dtype: int64

idade
10    116
12    107
11    102
13    100
9      96
14     85
15     63
8      57
16     52
17     48
7      14
18     12
19      5
20      2
21      1
Name: count, dtype: int64
idade
11    133
10    113
12    109
9     108
14     96
13     95
8      89
15     65
16     55
17     46
18     29
7      27
19     18
20     15
21      9
22      4
26      1
23      1
24      1
Name: count, dtype: int64
idade
12    148
11    132
10    128
13    103
9     102
15     98
14     97
16     80
8      68
17     59
18     26
21     25
7      20
19     20
20     20
22     17
23      9
24      2
27      1
25      1
Name: count, dtype: int64


### Ano

In [1315]:
col = 'ano'

year = 2022
dfs_yearly[year][col] = year

year = 2023
dfs_yearly[year][col] = year

year = 2024
dfs_yearly[year][col] = year

### Ano de nascimento

In [1316]:
col = 'ano_nascimento'

extract_data(
	col,
	col_2022 = 'Ano nasc',
	col_2023 = 'Data de Nasc',
	col_2024 = 'Data de Nasc'
)

In [1317]:
display_column_from_all(col)

0    2003
1    2005
2    2005
Name: ano_nascimento, dtype: int64

0    6/17/2015
1    5/31/2014
2    2/25/2016
Name: ano_nascimento, dtype: object

0   2016-07-28
1   2016-10-16
2   2016-08-16
Name: ano_nascimento, dtype: datetime64[ns]

In [1318]:
dfs_yearly[2022][col] = dfs_yearly[2022][col].astype('int32')

In [1319]:
dfs_yearly[2023][col] = pd.to_datetime(dfs_yearly[2023][col])
dfs_yearly[2023][col] = dfs_yearly[2023][col].dt.year

In [1320]:
dfs_yearly[2024][col] = dfs_yearly[2024][col].dt.year

In [1321]:
display_column_from_all(col)

0    2003
1    2005
2    2005
Name: ano_nascimento, dtype: int32

0    2015
1    2014
2    2016
Name: ano_nascimento, dtype: int32

0    2016
1    2016
2    2016
Name: ano_nascimento, dtype: int32

### INDE

In [1322]:
col = 'inde'

extract_data(
    col,
    col_2022 = 'INDE 22',
    col_2023 = 'INDE 2023',
    col_2024 = 'INDE 2024'
)

display_column_from_all('inde')

0    5.783
1    7.055
2    6.591
Name: inde, dtype: float64

0    9.31095
1    8.22120
2    5.92975
Name: inde, dtype: float64

0    7.611367
1    8.002867
2      7.9522
Name: inde, dtype: object

In [1323]:
dfs_yearly[2024][col].describe()

count        1092
unique       1055
top       INCLUIR
freq           38
Name: inde, dtype: object

In [1324]:
dfs_yearly[2024][col].value_counts()

inde
INCLUIR     38
8.002867     1
7.9522       1
7.156367     1
5.4442       1
            ..
4.78185      1
7.785233     1
8.27555      1
8.0853       1
7.611367     1
Name: count, Length: 1055, dtype: int64

In [1325]:
replace_value_with(
    dfs_yearly[2024],
    col,
    'INCLUIR',
    np.nan
)

In [1326]:
dfs_yearly[2024][col] = dfs_yearly[2024][col].astype(float)

In [1327]:
display_column_from_all(col)

0    5.783
1    7.055
2    6.591
Name: inde, dtype: float64

0    9.31095
1    8.22120
2    5.92975
Name: inde, dtype: float64

0    7.611367
1    8.002867
2    7.952200
Name: inde, dtype: float64

### Pedra 22

In [1328]:
col = 'pedra_22'

extract_data(
	col,
	col_2022 = 'Pedra 22',
	col_2023 = 'Pedra 22',
	col_2024 = 'Pedra 22'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0     Quartzo
1    Ametista
2       Ágata
Name: pedra_22, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_22, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_22, dtype: object

pedra_22
Ametista    348
Ágata       250
Quartzo     132
Topázio     130
Name: count, dtype: int64
pedra_22
Ametista    264
Ágata       165
Topázio     118
Quartzo      53
Name: count, dtype: int64
pedra_22
Ametista    218
Ágata       117
Topázio     106
Quartzo      31
Name: count, dtype: int64


### Pedra 23

In [1329]:
col = 'pedra_23'

extract_data(
	col,
	col_2022 = 'Pedra 23',
	col_2023 = 'Pedra 2023',
	col_2024 = 'Pedra 23'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

Failed to find column on 2022 | 'Pedra 23'
Failed to process text on df[pedra_23] | Can only use .str accessor with string values!


0   NaN
1   NaN
2   NaN
Name: pedra_23, dtype: float64

0    Topázio
1    Topázio
2    Quartzo
Name: pedra_23, dtype: object

0    NaN
1    NaN
2    NaN
Name: pedra_23, dtype: object

Series([], Name: count, dtype: int64)
pedra_23
Ametista    381
Agata       246
Topázio     232
Quartzo      72
Name: count, dtype: int64
pedra_23
Ametista    308
Topázio     187
Agata       153
Quartzo      42
Name: count, dtype: int64


### Pedra 24

In [1330]:
col = 'pedra_24'

extract_data(
	col,
	col_2022 = 'Pedra 24',
	col_2023 = 'Pedra 24',
	col_2024 = 'Pedra 2024'
)

display_column_from_all(col)
display_value_counts_from_all(col)

Failed to find column on 2022 | 'Pedra 24'
Failed to find column on 2023 | 'Pedra 24'


0   NaN
1   NaN
2   NaN
Name: pedra_24, dtype: float64

0   NaN
1   NaN
2   NaN
Name: pedra_24, dtype: float64

0    Ametista
1     Topázio
2    Ametista
Name: pedra_24, dtype: object

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
pedra_24
Ametista    391
Topázio     326
Agata       225
Quartzo     112
INCLUIR      38
Name: count, dtype: int64


In [1331]:
replace_value_with(
    dfs_yearly[2024],
    col,
    'INCLUIR'
)

In [1332]:
display_value_counts_from_all(
    col
)

Series([], Name: count, dtype: int64)
Series([], Name: count, dtype: int64)
pedra_24
Ametista    391
Topázio     326
Agata       225
Quartzo     112
Name: count, dtype: int64


### Gênero

In [1333]:
col = 'genero'

extract_data(
	col,
	col_2022 = 'Gênero',
	col_2023 = 'Gênero',
	col_2024 = 'Gênero'
)

display_column_from_all(col)
display_value_counts_from_all(col)

0    Menina
1    Menina
2    Menina
Name: genero, dtype: object

0     Feminino
1    Masculino
2    Masculino
Name: genero, dtype: object

0    Masculino
1     Feminino
2    Masculino
Name: genero, dtype: object

genero
Menina    457
Menino    403
Name: count, dtype: int64
genero
Feminino     546
Masculino    468
Name: count, dtype: int64
genero
Feminino     623
Masculino    533
Name: count, dtype: int64


In [1334]:
dfs_yearly[2022] = dfs_yearly[2022].replace(
    {
        'Menina':'Feminino',
        'Menino':'Masculino'
	}
)

In [1335]:
process_text_on_all_years(col)
display_column_from_all(col)

0    Feminino
1    Feminino
2    Feminino
Name: genero, dtype: object

0     Feminino
1    Masculino
2    Masculino
Name: genero, dtype: object

0    Masculino
1     Feminino
2    Masculino
Name: genero, dtype: object

### Ano de ingresso

In [1336]:
col = 'ano_ingresso'

extract_data(
	col,
	col_2022 = 'Ano ingresso',
	col_2023 = 'Ano ingresso',
	col_2024 = 'Ano ingresso'
)

display_column_from_all(col)
display_value_counts_from_all(col)

0    2016
1    2017
2    2016
Name: ano_ingresso, dtype: int64

0    2023
1    2023
2    2023
Name: ano_ingresso, dtype: int64

0    2024
1    2024
2    2024
Name: ano_ingresso, dtype: int64

ano_ingresso
2022    399
2019    139
2021    138
2018     67
2020     48
2017     40
2016     29
Name: count, dtype: int64
ano_ingresso
2023    369
2022    268
2021    137
2019     98
2018     46
2020     40
2017     32
2016     24
Name: count, dtype: int64
ano_ingresso
2024    349
2021    345
2023    248
2022    214
Name: count, dtype: int64


### Instituição de ensino

In [1337]:
col = 'instituicao_ensino'

extract_data(
	col,
	col_2022 = 'Instituição de ensino',
	col_2023 = 'Instituição de ensino',
	col_2024 = 'Instituição de ensino'
)

process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0    Escola pública
1      Rede decisão
2      Rede decisão
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

instituicao_ensino
Escola pública    752
Rede decisão      106
Escola jp ii        2
Name: count, dtype: int64
instituicao_ensino
Pública                                      809
Privada - programa de apadrinhamento         100
Privada *parcerias com bolsa 100%             60
Privada                                       28
Privada - pagamento por *empresa parceira      8
Concluiu o 3º em                               7
Nenhuma das opções acima                       2
Name: count, dtype: int64
instituicao_ensino
Pública                                      913
Privada - programa de apadrinhamento          96
Privada                                       76
Privada *parcerias com bolsa 100%             41
Bolsista universitário *formado (a)           13
Privada - pagamento por *empresa parceira      9
Concluiu o 3º em                               7
Name: count, dtype: int64


In [1338]:
dfs_yearly[2022] = dfs_yearly[2022].replace(
	{
        'Escola pública' : 'Pública',
	}
)

In [1339]:
process_text_on_all_years(col)
display_column_from_all(col)
display_value_counts_from_all(col)

0         Pública
1    Rede decisão
2    Rede decisão
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

0    Pública
1    Pública
2    Pública
Name: instituicao_ensino, dtype: object

instituicao_ensino
Pública         752
Rede decisão    106
Escola jp ii      2
Name: count, dtype: int64
instituicao_ensino
Pública                                      809
Privada - programa de apadrinhamento         100
Privada *parcerias com bolsa 100%             60
Privada                                       28
Privada - pagamento por *empresa parceira      8
Concluiu o 3º em                               7
Nenhuma das opções acima                       2
Name: count, dtype: int64
instituicao_ensino
Pública                                      913
Privada - programa de apadrinhamento          96
Privada                                       76
Privada *parcerias com bolsa 100%             41
Bolsista universitário *formado (a)           13
Privada - pagamento por *empresa parceira      9
Concluiu o 3º em                               7
Name: count, dtype: int64


### IAN

In [1340]:
col = 'ian'

extract_data(
	col,
	col_2022 = 'IAN',
	col_2023 = 'IAN',
	col_2024 = 'IAN'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0     5.0
1    10.0
2    10.0
Name: ian, dtype: float64

0    10.0
1     5.0
2    10.0
Name: ian, dtype: float64

0    10.0
1    10.0
2    10.0
Name: ian, dtype: float64

### IDA

In [1341]:
col = 'ida'

extract_data(
	col,
	col_2022 = 'IDA',
	col_2023 = 'IDA',
	col_2024 = 'IDA'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    4.0
1    6.8
2    5.6
Name: ida, dtype: float64

0    9.6
1    8.9
2    6.3
Name: ida, dtype: float64

0    8.0
1    8.0
2    8.0
Name: ida, dtype: float64

### IEG

In [1342]:
col = 'ieg'

extract_data(
	col,
	col_2022 = 'IEG',
	col_2023 = 'IEG',
	col_2024 = 'IEG'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    4.1
1    5.2
2    7.9
Name: ieg, dtype: float64

0    10.0
1     9.1
2     7.6
Name: ieg, dtype: float64

0    8.666667
1    9.333333
2    9.083333
Name: ieg, dtype: float64

### IAA

In [1343]:
col = 'iaa'

extract_data(
	col,
	col_2022 = 'IAA',
	col_2023 = 'IAA',
	col_2024 = 'IAA'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    8.3
1    8.8
2    0.0
Name: iaa, dtype: float64

0    9.5
1    8.5
2    0.0
Name: iaa, dtype: float64

0    10.002
1    10.002
2    10.002
Name: iaa, dtype: float64

### IPS

In [1344]:
col = 'ips'

extract_data(
	col,
	col_2022 = 'IPS',
	col_2023 = 'IPS',
	col_2024 = 'IPS'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    5.6
1    6.3
2    5.6
Name: ips, dtype: float64

0    8.13
1    8.14
2    3.14
Name: ips, dtype: float64

0    6.26
1    3.76
2    3.76
Name: ips, dtype: float64

### IPP

In [1345]:
col = 'ipp'

extract_data(
	col,
	col_2022 = 'IPP',
	col_2023 = 'IPP',
	col_2024 = 'IPP'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

Failed to find column on 2022 | 'IPP'


0   NaN
1   NaN
2   NaN
Name: ipp, dtype: float64

0    8.4375
1    7.5000
2    5.9375
Name: ipp, dtype: float64

0    5.625
1    7.500
2    7.500
Name: ipp, dtype: float64

### IPV

In [1346]:
col = 'ipv'

extract_data(
	col,
	col_2022 = 'IPV',
	col_2023 = 'IPV',
	col_2024 = 'IPV'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    7.278
1    6.778
2    7.556
Name: ipv, dtype: float64

0    8.920
1    8.585
2    6.260
Name: ipv, dtype: float64

0    5.446667
1    7.050000
2    7.046667
Name: ipv, dtype: float64

### Matemática

In [1347]:
col = 'nota_mat'

extract_data(
	col,
	col_2022 = 'Matem',
	col_2023 = 'Mat',
	col_2024 = 'Mat'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    2.7
1    6.3
2    5.8
Name: nota_mat, dtype: float64

0    9.8
1    8.5
2    7.0
Name: nota_mat, dtype: float64

0    10.0
1    10.0
2    10.0
Name: nota_mat, dtype: float64

### Português

In [1348]:
col = 'nota_port'

extract_data(
	col,
	col_2022 = 'Portug',
	col_2023 = 'Por',
	col_2024 = 'Por'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    3.5
1    4.5
2    4.0
Name: nota_port, dtype: float64

0    9.4
1    9.2
2    5.5
Name: nota_port, dtype: float64

0    6.0
1    6.0
2    6.0
Name: nota_port, dtype: float64

### Inglês

In [1349]:
col = 'nota_ing'

extract_data(
	col,
	col_2022 = 'Inglês',
	col_2023 = 'Ing',
	col_2024 = 'Ing'
)

# process_text_on_all_years(col)
display_column_from_all(col)
# display_value_counts_from_all(col)

0    6.0
1    9.7
2    6.9
Name: nota_ing, dtype: float64

0   NaN
1   NaN
2   NaN
Name: nota_ing, dtype: float64

0   NaN
1   NaN
2   NaN
Name: nota_ing, dtype: float64

## Mergendo dados

In [1350]:
for year in dfs_yearly:
    dfs_yearly[year] = dfs_yearly[year].set_index('aluno')
dfs_yearly[2022]

,idade,ano,ano_nascimento,inde,pedra_22,pedra_23,pedra_24,genero,ano_ingresso,instituicao_ensino,ian,ida,ieg,iaa,ips,ipp,ipv,nota_mat,nota_port,nota_ing
aluno,,,,,,,,,,,,,,,,,,,,
Aluno-1,19,2022,2003,5.783,Quartzo,NaN,NaN,Feminino,2016,Pública,5.0,4.0,4.1,8.3,5.6,NaN,7.278,2.7,3.5,6.0
Aluno-2,17,2022,2005,7.055,Ametista,NaN,NaN,Feminino,2017,Rede decisão,10.0,6.8,5.2,8.8,6.3,NaN,6.778,6.3,4.5,9.7
Aluno-3,17,2022,2005,6.591,Ágata,NaN,NaN,Feminino,2016,Rede decisão,10.0,5.6,7.9,0.0,5.6,NaN,7.556,5.8,4.0,6.9
Aluno-4,17,2022,2005,5.951,Quartzo,NaN,NaN,Masculino,2017,Rede decisão,10.0,5.0,4.5,8.8,5.6,NaN,5.278,2.8,3.5,8.7
Aluno-5,17,2022,2005,7.427,Ametista,NaN,NaN,Feminino,2016,Rede decisão,10.0,5.2,8.6,7.9,5.6,NaN,7.389,7.0,2.9,5.7
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aluno-856,7,2022,2015,8.398,Topázio,NaN,NaN,Masculino,2022,Pública,10.0,8.4,9.1,10.0,7.5,NaN,8.000,8.5,8.2,NaN
Aluno-857,9,2022,2013,8.154,Topázio,NaN,NaN,Feminino,2022,Pública,5.0,9.5,8.9,8.5,7.5,NaN,9.083,9.6,9.3,NaN
Aluno-858,7,2022,2015,7.523,Ametista,NaN,NaN,Masculino,2022,Pública,10.0,7.7,7.1,10.0,5.6,NaN,7.333,8.4,6.9,NaN


### Lidando com dados anuais

In [1351]:
data : dict[int, pd.DataFrame] = {}
data[2022] = dfs_yearly[2022].copy()
data[2023] = dfs_yearly[2023].copy()
data[2024] = dfs_yearly[2024].copy()

for year in data:
    data[year].columns = [f'{col}-{year}' for col in data[year].columns]

data[2022].head(3)

,idade-2022,ano-2022,ano_nascimento-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,genero-2022,ano_ingresso-2022,instituicao_ensino-2022,ian-2022,ida-2022,ieg-2022,iaa-2022,ips-2022,ipp-2022,ipv-2022,nota_mat-2022,nota_port-2022,nota_ing-2022
aluno,,,,,,,,,,,,,,,,,,,,
Aluno-1,19,2022,2003,5.783,Quartzo,NaN,NaN,Feminino,2016,Pública,5.0,4.0,4.1,8.3,5.6,NaN,7.278,2.7,3.5,6.0
Aluno-2,17,2022,2005,7.055,Ametista,NaN,NaN,Feminino,2017,Rede decisão,10.0,6.8,5.2,8.8,6.3,NaN,6.778,6.3,4.5,9.7
Aluno-3,17,2022,2005,6.591,Ágata,NaN,NaN,Feminino,2016,Rede decisão,10.0,5.6,7.9,0.0,5.6,NaN,7.556,5.8,4.0,6.9


In [1352]:
students : dict[int, set] = {}

for year in data:
    students[year] = set(data[year].index)

In [1353]:
# Alunos que estão presentes apenas nos registros de 2023 para frente
students[2023] = students[2023] - students[2022]
print(len(students[2023]))

# Alunos que estão presentes apenas nos registros de 2024 para frente
students[2024] = students[2024] - students[2022] - students[2023]
print(len(students[2024]))

414
387


#### Atualizando alunos de 2022 até 2024

In [1354]:
df_students_2022 : pd.DataFrame

df_students_2022 = pd.merge(
	left = data[2022],
	right = data[2023],
	on = 'aluno',
	how = 'left'
)

df_students_2022 = pd.merge(
	left = df_students_2022,
	right = data[2024],
	on = 'aluno',
	how = 'left'
)

df_students_2022.head(3)

,idade-2022,ano-2022,ano_nascimento-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,genero-2022,ano_ingresso-2022,instituicao_ensino-2022,...,ian-2024,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-1,19,2022,2003,5.783,Quartzo,NaN,NaN,Feminino,2016,Pública,...,10.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-2,17,2022,2005,7.055,Ametista,NaN,NaN,Feminino,2017,Rede decisão,...,10.0,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-3,17,2022,2005,6.591,Ágata,NaN,NaN,Feminino,2016,Rede decisão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [1355]:
df_students_2022.shape, len(students[2022])

((860, 60), 860)

#### Atualizando alunos de 2023 até 2024

In [1356]:
df_students_2023 : pd.DataFrame

df_students_2023 = pd.merge(
	left = data[2023].loc[list(students[2023])],
	right = data[2024],
	on = 'aluno',
	how = 'left'
)

df_students_2023.head(3)

,idade-2023,ano-2023,ano_nascimento-2023,inde-2023,pedra_22-2023,pedra_23-2023,pedra_24-2023,genero-2023,ano_ingresso-2023,instituicao_ensino-2023,...,ian-2024,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-979,8,2023,2015,8.1617,NaN,Topázio,NaN,Masculino,2023,Pública,...,5.0,8.25,9.522727,9.502,7.51,8.125000,7.335000,9.5,7.0,NaN
Aluno-971,8,2023,2015,7.0169,NaN,Ametista,NaN,Masculino,2023,Pública,...,5.0,6.75,7.909091,7.418,7.51,7.500000,6.085000,8.5,5.0,NaN
Aluno-1082,12,2023,2011,7.1617,NaN,Ametista,NaN,Feminino,2023,Pública,...,5.0,3.75,7.571429,7.502,10.00,7.083333,6.943333,2.0,5.5,NaN


In [1357]:
df_students_2023.shape, len(students[2023])

((414, 40), 414)

#### Pegando os alunos de 2024

In [1358]:
df_students_2024 : pd.DataFrame

df_students_2024 = data[2024].loc[list(students[2024])]

df_students_2024.head(3)

,idade-2024,ano-2024,ano_nascimento-2024,inde-2024,pedra_22-2024,pedra_23-2024,pedra_24-2024,genero-2024,ano_ingresso-2024,instituicao_ensino-2024,ian-2024,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024
aluno,,,,,,,,,,,,,,,,,,,,
Aluno-1416,12,2024,2012,7.375503,NaN,NaN,Ametista,Feminino,2024,Pública,5.0,7.750000,8.276515,8.502,7.51,7.916667,6.386667,6.5,9.0,NaN
Aluno-1581,16,2024,2008,6.978702,NaN,NaN,Agata,Masculino,2024,Pública,5.0,6.333333,7.834845,8.334,6.26,7.083333,7.386667,7.0,4.5,7.5
Aluno-1630,16,2024,2008,8.583133,NaN,NaN,Topázio,Masculino,2024,Privada - programa de apadrinhamento,10.0,7.333333,9.833333,8.334,7.51,8.250000,8.702000,9.0,9.0,4.0


In [1359]:
df_students_2024.shape, len(students[2024])

((387, 20), 387)

### Juntando as sheets

#### Padronizando as colunas

In [1360]:
cols = df_students_2022.columns

for df in [df_students_2023, df_students_2024]:
	for col in cols:
		if col not in df.columns:
			df[col] = np.nan

#### Concatenando

In [1361]:
df = pd.concat(
    [df_students_2022,
    df_students_2023]
)

df = pd.concat(
    [df,
     df_students_2024]
)

df.shape

(1661, 60)

In [1362]:
len(students[2022] | students[2023] | students[2024])

1661

In [1363]:
df.dtypes

idade-2022                 float64
ano-2022                   float64
ano_nascimento-2022        float64
inde-2022                  float64
pedra_22-2022               object
pedra_23-2022              float64
pedra_24-2022              float64
genero-2022                 object
ano_ingresso-2022          float64
instituicao_ensino-2022     object
ian-2022                   float64
ida-2022                   float64
ieg-2022                   float64
iaa-2022                   float64
ips-2022                   float64
ipp-2022                   float64
ipv-2022                   float64
nota_mat-2022              float64
nota_port-2022             float64
nota_ing-2022              float64
idade-2023                 float64
ano-2023                   float64
ano_nascimento-2023        float64
inde-2023                  float64
pedra_22-2023               object
pedra_23-2023               object
pedra_24-2023              float64
genero-2023                 object
ano_ingresso-2023   

## Checando união
Vamos ver se os alunos estão coerentes entre anos

In [1364]:
def get_column_across_years(col):
    return [f'{col}-{year}' for year in dfs_yearly]

In [1365]:
def columns_are_equal(df : pd.DataFrame, col_list : list):
	combinations : list[tuple] = [x for x in itertools.combinations(list(range(len(col_list))), 2)]
	# print(combinations)

	values = df[col_list]

	are_equal = pd.Series()
	count = 0

	for row in values.values:
		values_are_equal = True
		for combination in combinations:
			x = combination[0]
			y = combination[1]
			if row[x] == row[y]:
				pass
			else:
				values_are_equal = False
		
		are_equal[count] = values_are_equal
		count += 1


	are_equal.index = df.index
	return are_equal.all(), are_equal

#### Pedras

In [1366]:
cols = get_column_across_years('pedra_22')
columns_are_equal(df, cols)[0]

np.False_

In [1367]:
cols = ['pedra_23-2023', 'pedra_23-2024']
columns_are_equal(df, cols)[0]

np.False_

#### Ano de ingresso

In [1368]:
cols = get_column_across_years('ano_ingresso')

df_check = df[cols].dropna()

check, are_equal = columns_are_equal(df_check, cols)
check

np.False_

In [1369]:
df_check.loc[are_equal == False, cols].head(10)

,ano_ingresso-2022,ano_ingresso-2023,ano_ingresso-2024
aluno,,,
Aluno-1,2016.0,2016.0,2021.0
Aluno-2,2017.0,2017.0,2021.0
Aluno-7,2017.0,2017.0,2021.0
Aluno-12,2020.0,2020.0,2021.0
Aluno-13,2016.0,2016.0,2021.0
Aluno-27,2016.0,2016.0,2021.0
Aluno-31,2019.0,2019.0,2021.0
Aluno-35,2019.0,2019.0,2021.0
Aluno-36,2019.0,2019.0,2021.0


Futuramente, vamos utilizar o primeiro ano de ingresso disponível por aluno para lidar com essa disparidade

#### Gênero

In [1370]:
cols = get_column_across_years('genero')

df_check = df[cols].dropna()

check, are_equal = columns_are_equal(df_check, cols)
check

np.False_

In [1371]:
df_check.loc[are_equal == False, cols].head(10)

,genero-2022,genero-2023,genero-2024
aluno,,,
Aluno-62,Masculino,Feminino,Feminino
Aluno-64,Masculino,Feminino,Feminino
Aluno-78,Masculino,Feminino,Feminino
Aluno-534,Feminino,Masculino,Masculino
Aluno-603,Feminino,Masculino,Masculino
Aluno-629,Masculino,Feminino,Feminino
Aluno-719,Feminino,Masculino,Masculino
Aluno-777,Masculino,Feminino,Feminino
Aluno-784,Masculino,Feminino,Feminino


#### Ano de nascimento

In [1372]:
cols = get_column_across_years('ano_nascimento')

df_check = df[cols].dropna()

check, are_equal = columns_are_equal(df_check, cols)
check

np.False_

In [1373]:
df_check.loc[are_equal == False, cols].head(10)

,ano_nascimento-2022,ano_nascimento-2023,ano_nascimento-2024
aluno,,,
Aluno-27,2005.0,2006.0,2006.0
Aluno-158,2005.0,2009.0,2009.0
Aluno-161,2011.0,2009.0,2009.0
Aluno-178,2009.0,2009.0,2008.0
Aluno-184,2006.0,2010.0,2010.0
Aluno-444,2011.0,2011.0,2009.0
Aluno-454,2008.0,2008.0,2007.0


Vamos checar se as idades estão mais coerentes

In [1374]:
df[800:810][get_column_across_years('idade')]


,idade-2022,idade-2023,idade-2024
aluno,,,
Aluno-801,9.0,9.0,10.0
Aluno-802,9.0,10.0,11.0
Aluno-803,10.0,NaN,NaN
Aluno-804,10.0,11.0,NaN
Aluno-805,8.0,9.0,10.0
Aluno-806,8.0,9.0,10.0
Aluno-807,8.0,9.0,10.0
Aluno-808,8.0,9.0,10.0
Aluno-809,7.0,NaN,NaN


Vamos ajustar `ano_nascimento` para condizer com a idade.

In [1375]:
for year in dfs_yearly:
    dfs_yearly[year]['ano_nascimento'] = dfs_yearly[year]['ano'] - dfs_yearly[year]['idade']

dfs_yearly[2022].head(3)

,idade,ano,ano_nascimento,inde,pedra_22,pedra_23,pedra_24,genero,ano_ingresso,instituicao_ensino,ian,ida,ieg,iaa,ips,ipp,ipv,nota_mat,nota_port,nota_ing
aluno,,,,,,,,,,,,,,,,,,,,
Aluno-1,19,2022,2003,5.783,Quartzo,NaN,NaN,Feminino,2016,Pública,5.0,4.0,4.1,8.3,5.6,NaN,7.278,2.7,3.5,6.0
Aluno-2,17,2022,2005,7.055,Ametista,NaN,NaN,Feminino,2017,Rede decisão,10.0,6.8,5.2,8.8,6.3,NaN,6.778,6.3,4.5,9.7
Aluno-3,17,2022,2005,6.591,Ágata,NaN,NaN,Feminino,2016,Rede decisão,10.0,5.6,7.9,0.0,5.6,NaN,7.556,5.8,4.0,6.9


In [1376]:
for col_age, col_year, col_birth in zip(get_column_across_years('idade'), get_column_across_years('ano'), get_column_across_years('ano_nascimento')):
    df[col_birth] = df[col_year] - df[col_age]
df[get_column_across_years('ano_nascimento')].head()

,ano_nascimento-2022,ano_nascimento-2023,ano_nascimento-2024
aluno,,,
Aluno-1,2003.0,2003.0,2003.0
Aluno-2,2005.0,2005.0,2005.0
Aluno-3,2005.0,NaN,NaN
Aluno-4,2005.0,NaN,NaN
Aluno-5,2005.0,2005.0,NaN


## Dados únicos
Temos dados que não fazem sentidos serem registrados anualmente. Vamos padronizá-los e retirar os desnecessários.

In [1377]:
def make_column_with_earliest_information(df : pd.DataFrame, col : str):

	new_col = df[get_column_across_years(col)].copy()

	new_col[col] = new_col[f'{col}-2022']
	missing = new_col[col].isna()

	new_col.loc[missing, col] = new_col[f'{col}-2023']
	missing = new_col[col].isna()

	new_col.loc[missing, col] = new_col[f'{col}-2024']
	missing = new_col[col].isna()

	return new_col[col]

In [1378]:
def make_column_with_latest_information(df : pd.DataFrame, col : str):

	new_col = df[get_column_across_years(col)].copy()

	new_col[col] = new_col[f'{col}-2024']
	missing = new_col[col].isna()

	new_col.loc[missing, col] = new_col[f'{col}-2023']
	missing = new_col[col].isna()

	new_col.loc[missing, col] = new_col[f'{col}-2022']
	missing = new_col[col].isna()

	return new_col[col]

In [1379]:
new_cols = {}

#### Ano de ingresso

In [1380]:
standardized_column = 'ano_ingresso'

new_cols[standardized_column] = make_column_with_earliest_information(df, standardized_column)

display(new_cols[standardized_column].head(5))
new_cols[standardized_column].isna().sum()

aluno
Aluno-1    2016.0
Aluno-2    2017.0
Aluno-3    2016.0
Aluno-4    2017.0
Aluno-5    2016.0
Name: ano_ingresso, dtype: float64

np.int64(0)

In [1381]:
for col in get_column_across_years(standardized_column):
    df = df.drop(col, axis = 1)

In [1382]:
df[standardized_column] = new_cols[standardized_column].astype(int)
df.head(3)

,idade-2022,ano-2022,ano_nascimento-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,genero-2022,instituicao_ensino-2022,ian-2022,...,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024,ano_ingresso
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-1,19.0,2022.0,2003.0,5.783,Quartzo,NaN,NaN,Feminino,Pública,5.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016
Aluno-2,17.0,2022.0,2005.0,7.055,Ametista,NaN,NaN,Feminino,Rede decisão,10.0,...,NaN,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017
Aluno-3,17.0,2022.0,2005.0,6.591,Ágata,NaN,NaN,Feminino,Rede decisão,10.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016


#### Ano de nascimento

In [1383]:
standardized_column = 'ano_nascimento'

new_cols[standardized_column] = make_column_with_earliest_information(df, standardized_column)

display(new_cols[standardized_column].head(5))
print(new_cols[standardized_column].isna().sum())

for col in get_column_across_years(standardized_column):
    df = df.drop(col, axis = 1)
    
df[standardized_column] = new_cols[standardized_column].astype(int)
df.head(3)

aluno
Aluno-1    2003.0
Aluno-2    2005.0
Aluno-3    2005.0
Aluno-4    2005.0
Aluno-5    2005.0
Name: ano_nascimento, dtype: float64

0


,idade-2022,ano-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,genero-2022,instituicao_ensino-2022,ian-2022,ida-2022,...,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024,ano_ingresso,ano_nascimento
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-1,19.0,2022.0,5.783,Quartzo,NaN,NaN,Feminino,Pública,5.0,4.0,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,2003
Aluno-2,17.0,2022.0,7.055,Ametista,NaN,NaN,Feminino,Rede decisão,10.0,6.8,...,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,2005
Aluno-3,17.0,2022.0,6.591,Ágata,NaN,NaN,Feminino,Rede decisão,10.0,5.6,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,2005


#### Gênero

In [1384]:
standardized_column = 'genero'

new_cols[standardized_column] = make_column_with_latest_information(df, standardized_column)

display(new_cols[standardized_column].head(5))
print(new_cols[standardized_column].isna().sum())

for col in get_column_across_years(standardized_column):
    df = df.drop(col, axis = 1)
    
df[standardized_column] = new_cols[standardized_column]
df.head(3)

aluno
Aluno-1     Feminino
Aluno-2     Feminino
Aluno-3     Feminino
Aluno-4    Masculino
Aluno-5     Feminino
Name: genero, dtype: object

0


,idade-2022,ano-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,instituicao_ensino-2022,ian-2022,ida-2022,ieg-2022,...,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024,ano_ingresso,ano_nascimento,genero
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-1,19.0,2022.0,5.783,Quartzo,NaN,NaN,Pública,5.0,4.0,4.1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,2003,Feminino
Aluno-2,17.0,2022.0,7.055,Ametista,NaN,NaN,Rede decisão,10.0,6.8,5.2,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2017,2005,Feminino
Aluno-3,17.0,2022.0,6.591,Ágata,NaN,NaN,Rede decisão,10.0,5.6,7.9,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2016,2005,Feminino


In [1385]:
df[standardized_column].loc['Aluno-719']

'Masculino'

#### Ajustando a ordem

In [1386]:
current_order = list(df.columns)

for item in list(new_cols.keys()):
	current_order.remove(item)
current_order = list(new_cols.keys()) + current_order

current_order

['ano_ingresso',
 'ano_nascimento',
 'genero',
 'idade-2022',
 'ano-2022',
 'inde-2022',
 'pedra_22-2022',
 'pedra_23-2022',
 'pedra_24-2022',
 'instituicao_ensino-2022',
 'ian-2022',
 'ida-2022',
 'ieg-2022',
 'iaa-2022',
 'ips-2022',
 'ipp-2022',
 'ipv-2022',
 'nota_mat-2022',
 'nota_port-2022',
 'nota_ing-2022',
 'idade-2023',
 'ano-2023',
 'inde-2023',
 'pedra_22-2023',
 'pedra_23-2023',
 'pedra_24-2023',
 'instituicao_ensino-2023',
 'ian-2023',
 'ida-2023',
 'ieg-2023',
 'iaa-2023',
 'ips-2023',
 'ipp-2023',
 'ipv-2023',
 'nota_mat-2023',
 'nota_port-2023',
 'nota_ing-2023',
 'idade-2024',
 'ano-2024',
 'inde-2024',
 'pedra_22-2024',
 'pedra_23-2024',
 'pedra_24-2024',
 'instituicao_ensino-2024',
 'ian-2024',
 'ida-2024',
 'ieg-2024',
 'iaa-2024',
 'ips-2024',
 'ipp-2024',
 'ipv-2024',
 'nota_mat-2024',
 'nota_port-2024',
 'nota_ing-2024']

In [1387]:
df = df[current_order]
df

,ano_ingresso,ano_nascimento,genero,idade-2022,ano-2022,inde-2022,pedra_22-2022,pedra_23-2022,pedra_24-2022,instituicao_ensino-2022,...,ian-2024,ida-2024,ieg-2024,iaa-2024,ips-2024,ipp-2024,ipv-2024,nota_mat-2024,nota_port-2024,nota_ing-2024
aluno,,,,,,,,,,,,,,,,,,,,,
Aluno-1,2016,2003,Feminino,19.0,2022.0,5.783,Quartzo,NaN,NaN,Pública,...,10.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-2,2017,2005,Feminino,17.0,2022.0,7.055,Ametista,NaN,NaN,Rede decisão,...,10.0,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-3,2016,2005,Feminino,17.0,2022.0,6.591,Ágata,NaN,NaN,Rede decisão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-4,2017,2005,Masculino,17.0,2022.0,5.951,Quartzo,NaN,NaN,Rede decisão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Aluno-5,2016,2005,Feminino,17.0,2022.0,7.427,Ametista,NaN,NaN,Rede decisão,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Aluno-1476,2024,2013,Feminino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,10.0,8.500000,10.000000,9.502,5.635,8.437500,8.752500,8.00,9.0,NaN
Aluno-1629,2024,2007,Feminino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,7.250000,8.645503,9.585,7.510,7.187500,7.266000,8.75,3.0,10.0
Aluno-1574,2024,2008,Feminino,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,5.0,7.333333,8.399471,8.751,7.510,7.812500,8.616667,7.00,6.0,9.0


## Criando dados novos (WIP)

#### Desistência
Classificaremos como desistente o aluno que tinha dados no ano anterior, e não tem mais.

### Média dos índices
Média dos 3 anos

### IPP (2022)
Conforme orientado, iremos preencher o IPP faltando do ano 2022 via KNN.

## Atualizando bases anuais

In [1388]:
def get_columns_from_year(
		target_year : int,
		include_non_yearly : bool = True
		):
	target_year_string = f'{target_year}'
	year_cols = []

	for col in df.columns:
		elements = col.split('-')
		if len(elements) < 2:
			if include_non_yearly:
				year_cols.append(col)
		else:
			if elements[1] == target_year_string:
				year_cols.append(col)
			else:
				pass

	return year_cols

In [1389]:
def get_non_yearly_columns():
	fixed_cols = []

	for col in df.columns:
		elements = col.split('-')
		if len(elements) < 2:
			fixed_cols.append(col)
		else:
			pass

	return fixed_cols    

In [1408]:
dfs_yearly[2022] = df[get_columns_from_year(2022, False)].dropna(how = 'all')
dfs_yearly[2023] = df[get_columns_from_year(2023, False)].dropna(how = 'all')
dfs_yearly[2024] = df[get_columns_from_year(2024, False)].dropna(how = 'all')

In [1414]:
df_fixed_cols = df[get_non_yearly_columns()]
df_fixed_cols.head()

,ano_ingresso,ano_nascimento,genero
aluno,,,
Aluno-1,2016,2003,Feminino
Aluno-2,2017,2005,Feminino
Aluno-3,2016,2005,Feminino
Aluno-4,2017,2005,Masculino
Aluno-5,2016,2005,Feminino


In [1415]:
for year in dfs_yearly:
	dfs_yearly[year] = pd.merge(
		left = df_fixed_cols,
		right = dfs_yearly[year],
		on = 'aluno'
	)

# Exportando a base

In [1416]:
df.to_csv(FILE_JOINED_DATA)

dfs_yearly[2022].to_csv(FILE_JOINED_DATA2022)
dfs_yearly[2023].to_csv(FILE_JOINED_DATA2023)
dfs_yearly[2024].to_csv(FILE_JOINED_DATA2024)


dfs_yearly[2024].to_csv('test.csv')

---
# Conexão com o PostgreSQL

In [ ]:
def connect_to_db(
        dbname = 'datathon',
        user = 'postgres',
        password_path = FILE_POSTGRES_PW,
        host = 'localhost',
        port = '5432'
):
    password = ''
    with open(password_path) as f:
        password = f.read()

    conn = ps.connect(
        dbname = dbname,
        user = user,
        password = password,
        host = host,
        port = port
    )

    return conn

In [ ]:
cursor = None
conn = None
try:
    conn = connect_to_db()
    sql_create_schema = 'CREATE SCHEMA IF NOT EXISTS datathon;'
    
    cursor = conn.cursor()
    cursor.execute(sql_create_schema)
    conn.commit()

    print("Schema created successfully")
except Exception as e:
    
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Failed to create schema | connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



In [ ]:
sql_types = {
    'object' : 'VARCHAR(255)',
    'int64': 'INT',
    'float64': 'FLOAT'
}

schema_types = {}

for col in df.columns:
    schema_types[col] = sql_types[f'{df[col].dtype}']

schema_types

{'genero': 'VARCHAR(255)',
 'ano_ingresso': 'INT',
 'ano_nascimento': 'INT',
 'fase_2022': 'VARCHAR(255)',
 'ian_2022': 'FLOAT',
 'ida_2022': 'FLOAT',
 'ieg_2022': 'FLOAT',
 'iaa_2022': 'FLOAT',
 'ips_2022': 'FLOAT',
 'ipp_2022': 'FLOAT',
 'ipv_2022': 'FLOAT',
 'matematica_2022': 'FLOAT',
 'portugues_2022': 'FLOAT',
 'ingles_2022': 'FLOAT',
 'instituicao_2022': 'VARCHAR(255)',
 'fase_2023': 'VARCHAR(255)',
 'ian_2023': 'FLOAT',
 'ida_2023': 'FLOAT',
 'ieg_2023': 'FLOAT',
 'iaa_2023': 'FLOAT',
 'ips_2023': 'FLOAT',
 'ipp_2023': 'FLOAT',
 'ipv_2023': 'FLOAT',
 'matematica_2023': 'FLOAT',
 'portugues_2023': 'FLOAT',
 'ingles_2023': 'FLOAT',
 'instituicao_2023': 'VARCHAR(255)',
 'fase_2024': 'VARCHAR(255)',
 'ian_2024': 'FLOAT',
 'ida_2024': 'FLOAT',
 'ieg_2024': 'FLOAT',
 'iaa_2024': 'FLOAT',
 'ips_2024': 'FLOAT',
 'ipp_2024': 'FLOAT',
 'ipv_2024': 'FLOAT',
 'matematica_2024': 'FLOAT',
 'portugues_2024': 'FLOAT',
 'ingles_2024': 'FLOAT',
 'instituicao_2024': 'VARCHAR(255)'}

In [ ]:
sql_columns = ''
i = 0

for col in schema_types:
    col_type = schema_types[col]
    if i > 0:
        sql_columns += ', '
    sql_columns += f'{col} {col_type}'
    i += 1

sql_columns

'genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255)'

In [ ]:
sql_create_table = f'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos({sql_columns})'
sql_create_table

'DROP TABLE IF EXISTS datathon.passos_magicos; CREATE TABLE IF NOT EXISTS datathon.passos_magicos(genero VARCHAR(255), ano_ingresso INT, ano_nascimento INT, fase_2022 VARCHAR(255), ian_2022 FLOAT, ida_2022 FLOAT, ieg_2022 FLOAT, iaa_2022 FLOAT, ips_2022 FLOAT, ipp_2022 FLOAT, ipv_2022 FLOAT, matematica_2022 FLOAT, portugues_2022 FLOAT, ingles_2022 FLOAT, instituicao_2022 VARCHAR(255), fase_2023 VARCHAR(255), ian_2023 FLOAT, ida_2023 FLOAT, ieg_2023 FLOAT, iaa_2023 FLOAT, ips_2023 FLOAT, ipp_2023 FLOAT, ipv_2023 FLOAT, matematica_2023 FLOAT, portugues_2023 FLOAT, ingles_2023 FLOAT, instituicao_2023 VARCHAR(255), fase_2024 VARCHAR(255), ian_2024 FLOAT, ida_2024 FLOAT, ieg_2024 FLOAT, iaa_2024 FLOAT, ips_2024 FLOAT, ipp_2024 FLOAT, ipv_2024 FLOAT, matematica_2024 FLOAT, portugues_2024 FLOAT, ingles_2024 FLOAT, instituicao_2024 VARCHAR(255))'

In [ ]:
try:
    conn = connect_to_db()
    cursor = conn.cursor()

    # 4. Execute the CREATE TABLE statement
    cursor.execute(sql_create_table)

    # 5. Commit the transaction
    conn.commit()
    print("Table created successfully!")
except Exception as e:
    print(f'Failed to create schema | {e}')
    if conn is not None:
        conn.rollback()
finally:
    # 6. Close the cursor and connection
    if cursor is not None:
        cursor.close()
    if conn is not None:
        conn.close()

Failed to create schema | connection to server at "localhost" (::1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?
connection to server at "localhost" (127.0.0.1), port 5432 failed: Connection refused (0x0000274D/10061)
	Is the server running on that host and accepting TCP/IP connections?



## Subindo dados

In [ ]:
def ingest_data(file, table = 'datathon.passos_magicos'):
    conn = connect_to_db()
    cursor = conn.cursor()
    try:
        with open(file, 'r') as file:
            next(file) # skips header
            cursor.copy_expert(f'COPY {table} FROM STDIN WITH (FORMAT CSV)', file)
    except Exception as e:
        conn.rollback()
        print(f'Failed to insert | {e}')
    finally:
        conn.commit()
        if cursor is not None:
            cursor.close()
        if conn is not None:
            conn.close()

---

# Perguntas norteadoras

## 1. Adequação do nível (IAN): Qual é o perfil geral de defasagem dos alunos (IAN) e como ele evolui ao longo do ano?
(Ex.: quantos alunos estão moderadamente ou severamente defasados?)

## 2. Desempenho acadêmico (IDA): O desempenho acadêmico médio (IDA) está melhorando, estagnado ou caindo ao longo das fases e anos?

## 3. Engajamento nas atividades (IEG): O grau de engajamento dos alunos (IEG) tem relação direta com seus indicadores de desempenho (IDA) e do ponto de virada (IPV)?

## 4. Autoavaliação (IAA): As percepções dos alunos sobre si mesmos (IAA) são coerentes com seu desempenho real (IDA) e engajamento (IEG)?

## 5. Aspectos psicossociais (IPS): Há padrões psicossociais (IPS) que antecedem quedas de desempenho acadêmico ou de engajamento?

## 6. Aspectos psicopedagógicos (IPP): As avaliações psicopedagógicas (IPP) confirmam ou contradizem a defasagem identificada pelo IAN?

## 7. Ponto de virada (IPV): Quais comportamentos - acadêmicos, emocionais ou de engajamento - mais influenciam o IPV ao longo do tempo?

## 8. Multidimensionalidade dos indicadores: Quais combinações de indicadores (IDA + IEG + IPS + IPP) melhor explicam o desempenho global do aluno (INDE)?

## 9. Previsão de risco com Machine Learning: Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem? Construa um modelo preditivo que mostre uma probabilidade do aluno ou aluna entrar em risco de defasagem.

## 10. Efetividade do programa: Os indicadores mostram melhora consistente ao longo do ciclo nas diferentes fases (Quartzo, Ágata, Ametista e Topázio), confirmando o impacto real do programa?

## 11. Insights e criatividade: Você pode adicionar mais insights e pontos de vista não abordados nas perguntas, utilize a criatividade e a análise dos dados para trazer sugestões para a Passos Mágicos.

## Quais padrões nos indicadores permitem identificar alunos em risco antes de queda no desempenho ou aumento da defasagem?